# Test Module Act prompt from the Observation generator
In this notebook we will manually test some prompts and completions of the module Actuate. We are saving the results on a .txt file to compare them with other executions. 

## Experiment parameters
We will use the following parameters for the tests:

In [1]:
import sys
import os

# Add a description to the experiment if required, example:
current_folder = f"tests/experiment_notebooks/experiment_architecture"
# Scene path to be used in the experiment
scene_name = "scene_agent_takes_last_two"

args_substrate = "commons_harvest_open"

# Choose the world context to be used in the experiment, valid are those from data folder
args_world_context = "detailed_context"

scene_folder =  os.path.join(current_folder, f"scenes/{scene_name}")

# Choose the prompts folder to be used in the experiment, check for valid folders in prompts folder
prompts_folder = "base_prompts_v0"


## Experiment Code

In [2]:

# Add the parent directory to sys.path
sys.path.append(os.path.abspath('../../../'))

from dotenv import load_dotenv
import threading
from llm import LLMModels

from queue import Queue
import logging
from utils.llm import extract_answers
from utils.logging import CustomAdapter
from game_environment.substrates.python.commons_harvest_open import ASCII_MAP

#Change notebook root to the root of the project
os.chdir(os.path.abspath('../../../'))

# Save this current path in a variable for later use
current_path = os.getcwd()


2024-02-15 00:35:53.311307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 00:35:53.311378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 00:35:53.311411: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 00:35:53.998923: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datetime import datetime
import logging
import os
from dotenv import load_dotenv
import time
import traceback
from utils.logging import setup_logging, CustomAdapter
from game_environment.utils import generate_agent_actions_map, check_agent_out_of_game, get_defined_valid_actions
from agent.agent import Agent
from game_environment.server import start_server, get_scenario_map,  default_agent_actions_map
from llm import LLMModels
from utils.queue_utils import new_empty_queue
from utils.args_handler import get_args
from utils.files import extract_players
import json

pygame 2.5.2 (SDL 2.28.2, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:

def get_all_agents_observations(scene_folder:str) -> list[str]:
    """
    Load all the observations from the "agents_scenes_description.json" on  scene folder. 
    """
    agents_observations = []
    with open(scene_folder + "/agents_scenes_description.json", "r") as f:
        # Read it as a json dictionary
        agents_observations = json.load(f)
    
    agents_queue_observations = {agent_name: new_empty_queue() for agent_name in agents_observations.keys()}
    #For each agent in the agents_observations, we are creating a queue to store the observations
    for agent_name, agent_observations in agents_observations.items():
        agents_queue_observations[agent_name] = new_empty_queue()
        for observation in agent_observations:
            agents_queue_observations[agent_name].put(observation)
    return agents_queue_observations


get_all_agents_observations(scene_folder)

{'Pedro': <queue.Queue at 0x7fdda54d2b30>,
 'Laura': <queue.Queue at 0x7fde7c27ad10>,
 'Juan': <queue.Queue at 0x7fdda54d0310>}

#### Show what will be the input for the architecture flow.

In [5]:
# Print first observation of each agent
for agent_name, agent_observations in get_all_agents_observations(scene_folder).items():
    print(f"Agent: {agent_name} - First observation: {agent_observations.get()}")

Agent: Pedro - First observation: {'observation': '-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\nWWWWWWWWWWW\nFFFFF#FFFFF\nFFFFA2GFFFF', 'agents_in_observation': {'2': 'Laura'}, 'global_position': [8, 1], 'orientation': 3, 'last_observation': '-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\nWWWWWWWWWWW\nFFFFF#FFFFF\nFFFFA2GFFFF'}
Agent: Laura - First observation: {'observation': '-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\nWWWWWWWWWWW\nFFFF1GFFFFF\nFFFFA#GFFFF\nFFFGAGAAFFF', 'agents_in_observation': {'1': 'Pedro'}, 'global_position': [8, 2], 'orientation': 3, 'last_observation': '-----------\n-----------\n-----------\n-----------\n-----------\n-----------\n-----------\nWWWWWWWWWWW\nFFFFF1FFFFF\nFFFFA#GFFFF\nFFFGAGAAFFF'}
Agent: Juan - First observation: {'observation': 'FFFFFFFFFFF\nFFFFFFFFFFF\nFFFFFFFFFFG\nFFFFFFFFFAG\nFFFFFF

### Simulate architecture flow given an scene

In [8]:

from utils.files import load_config
from game_environment.scene_descriptor.observations_generator import ObservationsGenerator
import json

# Set up logging timestamp

logger_timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M-%S")
# load environment variables
load_dotenv(override=True)

logger = logging.getLogger(__name__)

# We are saving all the stream of printings in a file called "output.log"
folder_name_output = f'{current_folder}/results/{scene_name}___{logger_timestamp}'
if not os.path.exists(folder_name_output):
    os.makedirs(folder_name_output)
log_path = f'{folder_name_output}/output.log'
sys.stdout = open(log_path, 'w')


def game_loop(agents: list[Agent], scene_folder: str, substrate_name:str, scenario_info:dict[str]) -> None:
    """
    The main game loop. This will be run in a separate thread for each agent.
    """
    actions = None
    agents_scene_desc_queue = get_all_agents_observations(scene_folder)
    time_step = datetime.now().replace(minute=0, second=0, microsecond=0)
    date_format = load_config()['date_format']
    observationsGenerator = ObservationsGenerator(scenario_info['scenario_map'], [agent.name for agent in agents], substrate_name)
    # We are going to run the game loop until all the agents queues observations are empty

    while not all([agents_scene_desc_queue[agent_name].empty() for agent_name in agents_scene_desc_queue.keys()]):
        for agent in agents:
            agent_name = agent.name
            player_prefix = agent_name 
            logger.info("\n\n" + f"Agent's {agent.name} turn".center(50, '#') + "\n")

            if not agents_scene_desc_queue[agent_name].empty():
                scene_description = agents_scene_desc_queue[agent_name].get()
                scene_description['global_position'] = tuple(scene_description['global_position'])
                observations = observationsGenerator.get_observations_per_agent(scene_description, agent_name, True)
                
                state_changes = scene_description.get('observed_changes', [])
                game_time = time.strftime(date_format)
                agent_reward = 0
                step_actions = agent.move(observations, scene_description, state_changes, game_time, agent_reward)
                print(f"Agent: {agent_name} - Observation: {observations} - Actions: {step_actions}")

def main (logger):
    
    setup_logging(logger_timestamp)
    logger.info("Program started")
    start_time = time.time()
    
    mode = None # cooperative or None, if cooperative the agents will use the cooperative modules
    args_agents_bio_config = "no_bio"
    experiment_path = os.path.join("data", "defined_experiments", args_substrate)
    agents_bio_dir =  os.path.join( experiment_path, "agents_context", args_agents_bio_config)
    game_scenario = "default"
    players_context = [os.path.abspath(os.path.join(agents_bio_dir, player_file)) for player_file in os.listdir(agents_bio_dir)]

    players = extract_players(players_context)


    world_context_path = os.path.join(experiment_path, "world_context", f'{args_world_context}.txt')
    valid_actions = get_defined_valid_actions(game_name=args_substrate)
    scenario_obstacles  = ['W', '$'] # TODO : Change this. This should be also loaded from the scenario file
    scenario_info = {'scenario_map': get_scenario_map(game_name=args_substrate), 'valid_actions': valid_actions, 'scenario_obstacles': scenario_obstacles} ## TODO: ALL THIS HAVE TO BE LOADED USING SUBSTRATE NAME
    # Create agents
    agents = [Agent(name=player, data_folder="data", agent_context_file=player_context, world_context_file=world_context_path, scenario_info=scenario_info, mode=mode, prompts_folder=prompts_folder) for player, player_context in zip(players, players_context)]

    # Start the game server
    logger = CustomAdapter(logger, game_env=None)


    llm = LLMModels()
    try:
        game_loop(agents, scene_folder=scene_folder, substrate_name=args_substrate, scenario_info=scenario_info)
    except Exception as e:
        logger.exception("Exception: %s", e)


    # LLm total cost
    costs = llm.get_costs()
    tokens = llm.get_tokens()
    logger.info("LLM total cost: {:,.2f}, Cost by model: {}, Total tokens: {:,}, Tokens by model: {}".format(costs['total'], costs,  tokens['total'], tokens))

    end_time = time.time()
    logger.info("Execution time: %.2f minutes", (end_time - start_time)/60)

    logger.info("Program finished")
    
main(logger)



## Experiment Results

Management of the results will be done by filtering the results txt file by agent and then by module.

#### Save entire agent log

We are saving the whole agent log to a file for further analysis. It contains the entire flow of the agent, including the observations, prompts, completions and actions.

In [11]:
from tests.experiment_notebooks.experiment_architecture.experiment_utils import filter_log_by_agent, separate_modules_answers_from_log

# Uncomment next two lines if it did not work at first time, manually add the folder_name_output
results_name = "scene_agent_takes_last_two___2024-02-14--23-51-11"
folder_name_output = f"{current_folder}/results/{results_name}"
log_path = f'{folder_name_output}/output.log'
#read the log file
with open(log_path, 'r') as f:
    log = f.read()
_log_by_agent = filter_log_by_agent(log)['log_by_agent']

_log_by_agent

# For each agent create a folder and save each agent log in a file .log
for agent_name, agent_log in _log_by_agent.items():
    agent_folder = f"{folder_name_output}/{agent_name}"
    if not os.path.exists(agent_folder):
        os.makedirs(agent_folder)
    with open(f"{agent_folder}/{agent_name}.log", "w") as f:
        # Add to each line next string ########## NEW TURN ##########\n"
        agent_log = [f"\n########## NEW TURN ##########\n\n {log}" for log in agent_log]
        f.write('\n'.join(agent_log))

##### Filter by module each agent log

This cell will filter the results of the agent log by module and save them in a new file. 
Each agent will have a folder for each prompt, and that file will contain all the completions for that prompt.

In [12]:
from tests.experiment_notebooks.experiment_architecture.experiment_utils import separate_modules_answers_from_log
# Separate answers for each agent and then save the answers of each module in one file for module on the agent folder, separated by turn
for agent_name, agent_log in _log_by_agent.items():
    print(agent_name)
    agent_folder = f"{folder_name_output}/{agent_name}"
    print(agent_folder)
    promts_abs_path = os.path.abspath(f"{current_path}/prompts/{prompts_folder}")
    agent_answers = separate_modules_answers_from_log(f"{agent_folder}/{agent_name}.log", agent_name, promts_abs_path)
    
    for module, answers in agent_answers.items():
        folder = f"{agent_folder}/answer_per_module"
        if not os.path.exists(folder):
            os.makedirs(folder)
        with open(f"{folder}/{module}.log", "w") as f:
            for i, answer in enumerate(answers):
                f.write(f"\n\n########### ANSWER {i} ##########\n\n")
                f.write(answer)
                
answers_lens = {ans: len(agent_answers[ans]) for ans in agent_answers}
f"You got {answers_lens} answers for  module"

"You got {'react': 4, 'plan': 3, 'reflect_questions': 1, 'reflect_insight': 1, 'act': 3} answers for  module"